# Greeks Formulas:

## Changes:

- Use BSM formulas for Delta, Gamma, Theta, Vega instead of numerical approximation.
- Call and Put options have a MULTIPLIER of 100:
    - This means, for instance, if you have an ATM call, which has delta 0.5, you would break the Greek limit of 2000 by holding 40 = 2000/100 * 0.5 positions.
    - Underlying DOES NOT have this multiplier
    - Your strategy should just scale to this if you were using the non-multiplier limit previously

### Packages:

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.optimize import fsolve
from scipy.stats import norm

### BSM Set Up:

In [2]:
# Recall BSM Formulas:

def d1(S,K,T,r,sigma):
    return(np.log(S/K)+(r+sigma**2/2.)*T)/(sigma*np.sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*np.sqrt(T)

def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*np.exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
def bs_put(S,K,T,r,sigma):
    return K*np.exp(-r*T)-S+bs_call(S,K,T,r,sigma)

In [3]:
# Implied Volatility:

def iv_call(S,K,T,r,C):
    return max(0, fsolve((lambda sigma: np.abs(bs_call(S,K,T,r,sigma) - C)), [1])[0])
                      
def iv_put(S,K,T,r,P):
    return max(0, fsolve((lambda sigma: np.abs(bs_put(S,K,T,r,sigma) - P)), [1])[0])

### Defining the Greeks

- Remember $r = 0$ for this case, so we're not considering $\rho$ here. All formulas below assume that $r = 0$.
- Recall that for a share of the underlying, delta = 1, and the rest of the Greeks are zero
- Recall for a zero-coupon bond, all Greeks are zero (fixed payout)

In [4]:
def delta_call(S,K,T,C):
    sigma = iv_call(S,K,T,0,C)
    return 100 * norm.cdf(d1(S,K,T,0,sigma))

In [5]:
def gamma_call(S,K,T,C):
    sigma = iv_call(S,K,T,0,C)
    return 100 * norm.pdf(d1(S,K,T,0,sigma))/(S * sigma * np.sqrt(T))

In [6]:
def vega_call(S,K,T,C):
    sigma = iv_call(S,K,T,0,C)
    return 100 * norm.pdf(d1(S,K,T,0,sigma)) * S * np.sqrt(T)

In [7]:
def theta_call(S,K,T,C):
    sigma = iv_call(S,K,T,0,C)
    return 100 * S * norm.pdf(d1(S,K,T,0,sigma)) * sigma/(2 * np.sqrt(T))

In [8]:
def delta_put(S,K,T,C):
    sigma = iv_put(S,K,T,0,C)
    return 100 * (norm.cdf(d1(S,K,T,0,sigma)) - 1)

In [9]:
def gamma_put(S,K,T,C):
    sigma = iv_put(S,K,T,0,C)
    return 100 * norm.pdf(d1(S,K,T,0,sigma))/(S * sigma * np.sqrt(T))

In [10]:
def vega_put(S,K,T,C):
    sigma = iv_put(S,K,T,0,C)
    return 100 * norm.pdf(d1(S,K,T,0,sigma)) * S * np.sqrt(T)

In [11]:
def theta_put(S,K,T,C):
    sigma = iv_put(S,K,T,0,C)
    return 100 * S * norm.pdf(d1(S,K,T,0,sigma)) * sigma/(2 * np.sqrt(T))

### Example:
S = 100, K = 95, T = 25/252, call price = 10, put price = 1

In [12]:
# IV (note that the call and put are in unrelated markets, this is just to show how to use the formulas)
iv_call(100,95,25/252,0,10), iv_put(100,95,25/252,0,1)

(0.5908281373503532, 0.23054497261031975)

In [13]:
# Call values
vals1 = np.array([delta_call(100,95,25/252,10), gamma_call(100,95,25/252,10), theta_call(100,95,25/252,10), vega_call(100,95,25/252,10)])
vals1

array([6.43816394e+01, 2.00292017e+00, 3.49587572e+03, 1.17398968e+03])

In [14]:
# Put values
vals2 = np.array([delta_put(100,95,25/252,1), gamma_put(100,95,25/252,1), theta_put(100,95,25/252,1), vega_put(100,95,25/252,1)])
vals2

array([ -22.88370455,    4.16976207, 1108.13479467,  953.68817774])

In [15]:
# Say we want to do hold a 100 straddles as a trading strategy
# and we happen to (poorly) price the calls and puts as above
# The corresponding Greeks for the straddle would be:
vals_strat = (vals1 + vals2) * 100
vals_strat

array([  4149.79348254,    617.26822481, 460401.05144349, 212767.78559954])

In [16]:
# To test if our strategy is not breaking risk limits:
[vals_strat[0] < 2000, vals_strat[1] < 5000, vals_strat[2] < 1000000, vals_strat[3] < 500000]

[False, True, True, True]

We broke our delta limit, so the strategy isn't viable